# arXiv Monthly Submissions & Downloads

### Caveats
Here are some caveats to consider, taken from the [arXiv website](https://arxiv.org).
> _While we have attempted to extract download data representing unique full-text downloads by real users, there are many factors which affect accuracy. These factors include:_
    >>1. _the data is from the main [arXiv site](https://arxiv.org) and the arXiv mirrors, though some mirror data is incomplete;_
    >>2. _only web downloads are included (and not FTP or email "downloads" that were formerly supported);_
    >>3. _we have counted downloads according to the COUNTER algorithm which excludes rapid repeat downloads;_
    >>4. _we have attempted to identify and remove robot or automated downloads from the count (false positives lead to undercounting, failing to identify robots leads to overcounting);_
    >>5. _data prior to 2009 has not been cleaned with as much care as later data, it shows trends nonetheless._

In [1]:
from warnings import filterwarnings
filterwarnings('ignore')

# The usual suspects ...
import time_series as ts
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels

# And their accomplices ...
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA

# Plotting settings
sns.set(context='notebook', style='whitegrid', palette='deep', rc=None)

In [2]:
# Getting the data
downloads = pd.read_csv(
    'get_monthly_downloads_extracted_05-04-2018.csv')
submissions = pd.read_csv(
    'get_monthly_submissions_extracted-05-04-2018.csv')
data = pd.merge(submissions, downloads, on='month')

# Create a new file with the combined data
data.to_csv(
    'combined_submissions_and_downloads.csv', 
    sep=',')

Inspecting the dataset.

In [3]:
# Top 10 rows
data.head(10)

,month,submissions,historical_delta,downloads
0,1994-01,582,-28,0
1,1994-02,634,-19,1863
2,1994-03,729,-24,7048
3,1994-04,699,-39,6814
4,1994-05,823,-18,9601
5,1994-06,856,-32,11007
6,1994-07,840,-23,9488
7,1994-08,746,-20,10789
8,1994-09,851,-33,12872
9,1994-10,909,-23,16260


In [4]:
# Description
data.describe()

,submissions,historical_delta,downloads
count,290.000000,290.000000,2.900000e+02
mean,4662.703448,-6.575862,3.709525e+06
std,2803.957369,11.616244,4.453700e+06
min,582.000000,-49.000000,0.000000e+00
25%,2428.500000,-7.000000,6.735292e+05
50%,4112.500000,0.000000,2.067751e+06
75%,6726.000000,0.000000,4.982482e+06
max,11627.000000,0.000000,1.943365e+07


In [5]:
# Missing data
data.isnull().sum()

month               0
submissions         0
historical_delta    0
downloads           0
dtype: int64

| Columns | Description | Data Type | Action |
| :---: | :---: | :---: | :---: |
| `month` | month of activity | `string` | change to `datetime` |
| `submissions` | number of submissions made on a particular month | `integer` | do nothing |
| `historical_delta` | _undefined_ | `integer` | do nothing |
| `downloads` | number of downloads made on a particular month | `integer` | do nothing |
 
Our dataset has 4 columns and 290 rows. There are no missing values. We'll now proceed with further cleaning of our data.

In [ ]:
# Changing month to 'datetime'
ts.convert_to_datetime(data=data, column='month')
data.dtypes

We have successfully changed the `month` into a _datetime_ object.

***

### Number of submissions and downloads

In [ ]:
# Number of submissions vs. number of downloads
filterwarnings('ignore')
sns.jointplot(data=data, x='submissions', y='downloads', color='blue', kind='reg', size=13, )

Plotting the number of submissions against downloads, we find a strong linear correlation of $.91$ between the two variables. This tells us that as the number of submissions increases, so does the number of downloads.

From the scatterplot, we also find that submissions are more densely distributed in the range of $1 - 8000$. While on the other hand, downloads are more densely distributed in the range of $0 - 7.5$ million.

***

### Historical Delta ($h_\delta$)

In [ ]:
# Creating the time series
historical_delta = ts.create_series(data=data, time_column='month', category_column='historical_delta')

# Historical delta over time
ts.plot_series(historical_delta, title='Historical Delta ($h_{\delta}$) Over Time', ylabel='Historical Delta')

#### Stationarity of Historical Delta ($h_{\delta}$)

In [ ]:
ts.test_stationarity(historical_delta)

#### Summary

**insert summary here**

The _historical delta_ does not seem to show and trend or seasonality overtime. There are severe flactuations in the earlier years between $1994 - 2003$. After this period, the values are flatter along $0$.

Results from the Dickey-Fuller test show a test statistic value of $-2.27$, which is close to the critical values, further justifying a lack of trend.

***

### Submissions

In [ ]:
# Creating the submissions time series
submissions = ts.create_series(data=data, time_column='month', category_column='submissions')

# Number of submissions over time
ts.plot_series(submissions, title='Number of Submissions Over Times', ylabel='Submissions')

There has been a general upward trend in the number of submissions from $1994 - 2017$. There are relatively fewer flactuations in the period between $1994 - 2002$ compared to the period between $2003$ to the present date.

#### Stationarity of Submissions

In [ ]:
ts.test_stationarity(submissions)

We clearly have a rising trend over time. _This is not a stationary time series_. This can be observed from the rising mean, even though the standard deviation remains low.

We will now try to reducing the trend using logarithms. The intention is to penalize higher values more than lower values resulting in a flatter or less steeep trend.

In [ ]:
# Reducing the trend by applying logarithms
submissions_log = np.log(submissions)

# Testing for stationarity
ts.test_stationarity(submissions_log)

The logarithms have tried to remove some of the steep trend and has also made the standard much more flat. However, closer inspection shows that we still have a rising trend

In [ ]:
# Removing the rolling mean
submissions_log_diff = submissions_log - submissions_log.rolling(window=12, center=False).mean().dropna()
submissions_log_diff.dropna(inplace=True)
ts.test_stationarity(submissions_log_diff)

In [ ]:
# Exponentially weighted moving average
ewa = submissions_log.ewm(halflife=12).mean()
ts.plot_series(data=submissions_log, 
               data2=ewa, 
               color='blue', 
               title='Exponentially Weighted Moving Average', 
               ylabel='Submissions')

In [ ]:
# Removing the exponentially weighted mean from the series
submissions_log_ewa = submissions_log - ewa
submissions_log_ewa.dropna(inplace=True)
ts.test_stationarity(submissions_log_ewa)

In [ ]:
# Differencing
submissions_log_diff_shift = submissions_log - submissions_log.shift()
ts.plot_series(submissions_log_diff_shift, title='Differencing', ylabel='Submissions')

In [ ]:
submissions_log_diff_shift.dropna(inplace=True)
ts.test_stationarity(submissions_log_diff_shift)

In [ ]:
# Decomposing
decomposition = seasonal_decompose(submissions_log)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

# Plotting
plt.figure(figsize=(22, 12))
# Original
plt.subplot(411)
plt.plot(submissions_log, color='blue')
plt.title('Original', fontsize=18)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
# Trend
plt.subplot(412)
plt.plot(trend, color='blue')
plt.title('Trend', fontsize=18)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
# Seasonal
plt.subplot(413)
plt.plot(seasonal, color='blue')
plt.title('Seasonal', fontsize=18)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
# Residual
plt.subplot(414)
plt.plot(residual, color='blue')
plt.title('Residual', fontsize=18)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()

In [ ]:
# Checking stationarity of residuals
submissions_log_decompose_res = residual
submissions_log_decompose_res.dropna(inplace=True)
ts.test_stationarity(submissions_log_decompose_res)

In [ ]:
# Forecasting
# ACF and PACF plots:
lag_acf = acf(submissions_log_diff_shift, nlags=20)
lag_pacf = pacf(submissions_log_diff_shift, nlags=20, method='ols')

# Plotting ACF
plt.figure(figsize=(22, 10))
plt.subplot(121)
plt.plot(lag_acf, color='blue')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.axhline(y=0, linestyle='--', color='gray')
plt.axhline(y=-1.96/np.sqrt(len(submissions_log)), linestyle='--', color='gray')
plt.axhline(y=1.96/np.sqrt(len(submissions_log)), linestyle='--', color='gray')
plt.title('Autocorrelation Function', fontsize=24)

# Plotting PACF
plt.subplot(122)
plt.plot(lag_pacf, color='blue')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.axhline(y=0, linestyle='--', color='gray')
plt.axhline(y=-1.96/np.sqrt(len(submissions_log)), linestyle='--', color='gray')
plt.axhline(y=1.96/np.sqrt(len(submissions_log)), linestyle='--', color='gray')
plt.title('Partial Autocorrelation Function', fontsize=24)
plt.tight_layout()

In [ ]:
# AR Model
filterwarnings('ignore')
model = ARIMA(submissions_log_diff_shift, order=(1, 2, 0))
results_AR = model.fit(disp=-1)

# Plotting
ts.plot_series(data=submissions_log_diff_shift, 
               data2=results_AR.fittedvalues,
               ylabel='Submissions', 
               title='RSS: {}'.format(sum((results_AR.fittedvalues - submissions_log_diff_shift).fillna(0))**2))

In [ ]:
print(results_AR.summary())

In [ ]:
# MA Model
filterwarnings('ignore')
model = ARIMA(submissions_log, order=(0, 1, 2))
results_MA = model.fit(disp=-1)

# Plotting
ts.plot_series(data=submissions_log_diff_shift, 
               data2=results_MA.fittedvalues,
               ylabel='Submissions', 
               title='RSS: {}'.format(sum((results_MA.fittedvalues - submissions_log_diff_shift).fillna(0))**2))

In [ ]:
print(results_MA.summary())

In [ ]:
# ARIMA Model
filterwarnings('ignore')
model = ARIMA(submissions_log, order=(1, 1, 2))
results_ARIMA = model.fit(disp=-1)

# Plotting
ts.plot_series(data=submissions_log_diff_shift, 
               data2=results_ARIMA.fittedvalues,
               ylabel='Submissions', 
               title='RSS: {}'.format(sum((results_ARIMA.fittedvalues - submissions_log_diff_shift).fillna(0))**2))

In [ ]:
print(results_ARIMA.summary())

In [ ]:
# Reversing to the original scale
predictions_MA_diff = pd.Series(results_MA.fittedvalues, copy=True)

# Cumulative sum
predictions_MA_diff_cumsum = predictions_MA_diff.cumsum()

# Adding the base number
predictions_MA_log = pd.Series(submissions_log.ix[0], index=submissions_log.index)
predictions_MA_log = predictions_MA_log.add(predictions_MA_diff_cumsum, fill_value=0)

# Taking the exponent
predictions_MA = np.exp(predictions_MA_log)

# Plotting
ts.plot_predictions(data=submissions, 
                    data2=predictions_MA, 
                    ylabel='Submissions', 
                    title='RMSE {}'.format(np.sqrt(sum((predictions_MA - submissions)**2)/len(submissions))))

### Summary

**insert summary here**

***

### Downloads

In [ ]:
# Creating the time series
downloads = ts.create_series(data=data, time_column='month', category_column='downloads')

# Number of downloads over time
ts.plot_series(downloads, title='Number of Downloads Over Time', ylabel='Downloads')

Over time, downloads show a rising trend from $1995$ and gained a lot of momentum in $2012$. The years after $2012$ show significant increases, however there is a decrease in $2016$. After the drop in $2016$, the number of downloads rises even higher reaching a peak of close to 2 million downloads towards the end of $2017$.

#### Stationarity of Downloads

In [ ]:
ts.test_stationarity(downloads)

The Dickey-Fuller test shows a high test statistic relative to the sub-zero critical values. This means the number of downloads have a rolling mean that rises over time even though the standard deviation remains relatively low.

We also try to remove the trend in the downloads data by apply logarithms.

In [ ]:
# Removing seasonality and trend
downloads_log = np.log(downloads)
downloads_log = downloads_log.replace(np.negative(np.inf), 0)
downloads_log.dropna(inplace=True)
ts.test_stationarity(downloads_log)

In [ ]:
# Removing the rolling mean
downloads_log_diff = downloads_log - downloads_log.rolling(window=12, center=False).mean().dropna()
downloads_log_diff.dropna(inplace=True)
ts.test_stationarity(downloads_log_diff)

In [ ]:
# Exponentially weighted moving average
ewa = downloads_log.ewm(halflife=12).mean()

# Plotting
ts.plot_series(data=downloads_log, 
               data2=ewa, 
               color='blue', 
               title='Exponentially Weighted Moving Average', 
               ylabel='Downloads')

In [ ]:
# Removing the exponentially weighted mean from the series
downloads_log_ewa = downloads_log - ewa
downloads_log_ewa.dropna(inplace=True)
ts.test_stationarity(downloads_log_ewa)

In [ ]:
# Differencing
downloads_log_diff_shift = downloads_log - downloads_log.shift()
ts.plot_series(downloads_log_diff_shift, title='Differencing', ylabel='Downloads')

In [ ]:
downloads_log_diff_shift.dropna(inplace=True)
ts.test_stationarity(downloads_log_diff_shift)

In [ ]:
# Decomposing
decomposition = seasonal_decompose(downloads_log)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

# Plotting
plt.figure(figsize=(22, 12))
# Original
plt.subplot(411)
plt.plot(downloads_log, color='blue')
plt.title('Original', fontsize=18)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
# Trend
plt.subplot(412)
plt.plot(trend, color='blue')
plt.title('Trend', fontsize=18)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
# Seasonal
plt.subplot(413)
plt.plot(seasonal, color='blue')
plt.title('Seasonal', fontsize=18)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
# Residual
plt.subplot(414)
plt.plot(residual, color='blue')
plt.title('Residual', fontsize=18)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()

In [ ]:
# Checking stationarity of residuals
downloads_log_decompose_res = residual
downloads_log_decompose_res.dropna(inplace=True)
ts.test_stationarity(downloads_log_decompose_res)

In [ ]:
# Forecasting
# ACF and PACF plots:
lag_acf = acf(downloads_log_diff_shift, nlags=20)
lag_pacf = pacf(downloads_log_diff_shift, nlags=20, method='ols')

# Plotting ACF
plt.figure(figsize=(22, 10))
plt.subplot(121)
plt.plot(lag_acf, color='blue')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.axhline(y=0, linestyle='--', color='gray')
plt.axhline(y=-1.96/np.sqrt(len(downloads_log)), linestyle='--', color='gray')
plt.axhline(y=1.96/np.sqrt(len(downloads_log)), linestyle='--', color='gray')
plt.title('Autocorrelation Function', fontsize=24)

# Plotting PACF
plt.subplot(122)
plt.plot(lag_pacf, color='blue')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.axhline(y=0, linestyle='--', color='gray')
plt.axhline(y=-1.96/np.sqrt(len(downloads_log)), linestyle='--', color='gray')
plt.axhline(y=1.96/np.sqrt(len(downloads_log)), linestyle='--', color='gray')
plt.title('Partial Autocorrelation Function', fontsize=24)
plt.tight_layout()

In [ ]:
# AR Model
filterwarnings('ignore')
model = ARIMA(downloads_log_diff_shift, order=(1, 1, 0))
results_AR = model.fit(disp=-1)

# Plotting
ts.plot_series(data=downloads_log_diff_shift, 
               data2=results_AR.fittedvalues,
               ylabel='Downloads', 
               title='RSS: {}'.format(sum((results_AR.fittedvalues - downloads_log_diff_shift).fillna(0))**2))

In [ ]:
print(results_AR.summary())

In [ ]:
# MA Model
filterwarnings('ignore')
model = ARIMA(downloads_log, order=(0, 1, 1))
results_MA = model.fit(disp=-1)

# Plotting
ts.plot_series(data=downloads_log_diff_shift, 
               data2=results_MA.fittedvalues,
               ylabel='Downloads', 
               title='RSS: {}'.format(sum((results_MA.fittedvalues - downloads_log_diff_shift).fillna(0))**2))

In [ ]:
print(results_MA.summary())

In [ ]:
# ARIMA Model
filterwarnings('ignore')
model = ARIMA(downloads_log, order=(1, 1, 1))
results_ARIMA = model.fit(disp=-1)

# Plotting
ts.plot_series(data=downloads_log_diff_shift, 
               data2=results_ARIMA.fittedvalues,
               ylabel='Downloads',
               title='RSS: {}'.format(sum((results_ARIMA.fittedvalues - downloads_log_diff_shift).fillna(0))**2))

In [ ]:
print(results_ARIMA.summary())

In [ ]:
# By MA
# Reversing to the original scale
predictions_MA_diff = pd.Series(results_MA.fittedvalues, copy=True)

# Cumulative sum
predictions_MA_diff_cumsum = predictions_MA_diff.cumsum()

# Adding the base number
predictions_MA_log = pd.Series(downloads_log.ix[0], index=downloads_log.index)
predictions_MA_log = predictions_MA_log.add(predictions_MA_diff_cumsum, fill_value=0)

# Taking the exponent
predictions_MA = np.exp(predictions_MA_log)

# Plotting
ts.plot_predictions(data=downloads, 
                    data2=predictions_MA, 
                    ylabel='Downloads', 
                    title='RMSE {}'.format(np.sqrt(sum((predictions_MA - downloads)**2)/len(downloads))))

### Summary

**insert summary here**

***